In [5]:
!pip install rdkit
!pip install scikit-learn
!pip install lightgbm
!pip install xgboost
!pip install joblib


  Using cached numpy-1.24.4-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.1.1 requires sentencepiece, which is not installed.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.4 which is incompatible.
lifelines 0.30.0 requires pandas>=2.1, but you have pandas 2.0.3 which is incompatible.


In [ ]:
!pip install tensorflow

In [10]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors, rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from xgboost import XGBRegressor
import joblib
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

ImportError: cannot import name 'InconsistentVersionWarning' from 'sklearn.exceptions' (C:\Users\gagno\anaconda3\lib\site-packages\sklearn\exceptions.py)

In [ ]:
mol = Chem.MolFromSmiles('CCCC')
mol

In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np

# RDKit for SMILES parsing
from rdkit import Chem
from rdkit.Chem import AllChem

# For train-test split, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# For feature scaling
from sklearn.preprocessing import StandardScaler

# Simple neural network regressor from scikit-learn
from sklearn.neural_network import MLPRegressor

def smiles_to_morgan_fp(smi, radius=2, n_bits=1024):
    """
    Convert a SMILES string to a Morgan fingerprint (ECFP) vector of length n_bits.
    Returns a NumPy array of shape (n_bits,) or None if SMILES is invalid.
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    for i in range(n_bits):
        if fp.GetBit(i):
            arr[i] = 1
    return arr

def main():
    # 1. Load your CSV dataset (adjust filename/column names as needed)
    file_path = 'C:/Users/gagno/Downloads/molecule_dataset.csv'
    df = pd.read_csv(file_path)
    print(df.head(18))  # CSV should include 'Smiles' and 'Enhancement Factor'
    
    # 2. Basic cleaning: handle infinities and ensure 'Enhancement Factor' is numeric
    df = df.replace([np.inf, -np.inf], np.nan)
    df["Enhancement Factor"] = pd.to_numeric(df["Enhancement Factor"], errors='coerce')
    
    # Drop rows with missing values in 'Smiles' or 'Enhancement Factor'
    df.dropna(subset=["Smiles", "Enhancement Factor"], inplace=True)
    
    # 3. Convert SMILES to numeric features (Morgan fingerprints)
    smiles_list = df["Smiles"].astype(str).tolist()
    enhancement_factors = df["Enhancement Factor"].values
    
    X_fps = []
    valid_y = []
    for smi, val in zip(smiles_list, enhancement_factors):
        fp = smiles_to_morgan_fp(smi, radius=2, n_bits=1024)
        if fp is not None:
            X_fps.append(fp)
            valid_y.append(val)
    
    X = np.array(X_fps, dtype=float)
    y = np.array(valid_y, dtype=float)
    
    # 4. Train-test split
    #    We'll do 80-20 here, but with only 16 rows, consider cross-validation.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # 5. Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    
    # 6. Define and train a neural network (MLPRegressor)
    nn_model = MLPRegressor(
        hidden_layer_sizes=(16, 8),  # Example architecture
        activation='relu',
        solver='adam',
        max_iter=2000,
        random_state=42
    )
    nn_model.fit(X_train_scaled, y_train)
    
    # 7. Evaluate on the test set
    y_pred = nn_model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Adjusted R² = 1 - (1 - R²)*(n - 1)/(n - p - 1)
    n = len(y_test)
    p = X_test_scaled.shape[1]
    if n > p + 1:
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    else:
        adjusted_r2 = float('nan')
    
    print("\n=== Neural Network Model (Test Set) ===")
    print("Architecture: hidden_layer_sizes=(16, 8), activation='relu'")
    print(f"MSE  : {mse:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"MAE  : {mae:.4f}")
    print(f"R²   : {r2:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    
    # 8. Interactive input for new SMILES outside the dataset
    print("\n=== Predict Enhancement Factor for a new SMILES ===")
    while True:
        user_input = input("Enter a SMILES string (or type 'quit' to exit): ").strip()
        if user_input.lower() == 'quit':
            print("Exiting.")
            break
        
        new_fp = smiles_to_morgan_fp(user_input, radius=2, n_bits=1024)
        if new_fp is None:
            print("Invalid SMILES entered. Please try again.\n")
            continue
        
        # Scale the new fingerprint
        new_fp_scaled = scaler.transform([new_fp])
        pred_val = nn_model.predict(new_fp_scaled)[0]
        print(f"Predicted Enhancement Factor: {pred_val:.4f}\n")

if __name__ == "__main__":
    main()



    ID                         Name  \
0     1         Ibuprofen / HP-B-CD   
1     2      Dexamethasone / B - CD   
2     3         Piroxicam / HP-B-CD   
3     4    Hydrocortisone / HP-B-CD   
4     5       Albendazole / RM-B-CD   
5     6            Valtarsan / B-CD   
6     7        Nifedipine / HP-B-CD   
7     8        Nimodipine / HP-B-CD   
8     9     Carbamazepine / HP-B-CD   
9    10      Itraconazole / HP-B-CD   
10   11       Simvastatin / HP-B-CD   
11   12          Curcumin / HP-B-CD   
12   13  Indomethacin / HP- B - CD    
13   14     Naproxen / HP - B - CD    
14   15        Diclofenac / HP-B-CD   
15   16   Resveratrol / HP-Gamma CD   

                                               Smiles MolWeight  LogP  \
0                          CC(C)Cc1ccc(cc1)C(C)C(=O)O   1606.29  3.84   
1   CC(C)C1C(=O)CC2C(C1O)(CC(C1=CC(=O)C=C2C1(C)O)F...   1527.47  1.93   
2                     CN1NS(=O)(=O)C2=C(O)C=CN=C2C1=O   1466.35  2.20   
3              CC(=O)C1(CCC2C1(CCC1C(C2CC(C1

[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
[22:54:43] DEPRECATION WARNING: please use MorganGenerator
C:\Users\gagno\anaconda3\lib\site-packages\sklearn\neura


=== Neural Network Model (Test Set) ===
Architecture: hidden_layer_sizes=(16, 8), activation='relu'
MSE  : 4468340.3518
RMSE : 2113.8449
MAE  : 1908.4113
R²   : -71.0034
Adjusted R²: nan

=== Predict Enhancement Factor for a new SMILES ===
Enter a SMILES string (or type 'quit' to exit): quit
Exiting.


Above is the test model for NN. Below represents a fine tuned model:

In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np

# RDKit for SMILES parsing
from rdkit import Chem
from rdkit.Chem import AllChem

# For train-test split, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# For feature scaling
from sklearn.preprocessing import StandardScaler

# Simple neural network regressor from scikit-learn
from sklearn.neural_network import MLPRegressor

def smiles_to_morgan_fp(smi, radius=2, n_bits=256):
    """
    Convert a SMILES string to a Morgan fingerprint (ECFP) vector of length n_bits.
    Returns a NumPy array of shape (n_bits,) or None if SMILES is invalid.

    Reduced from 1024 to 256 bits to help avoid overfitting on a small dataset.
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    for i in range(n_bits):
        if fp.GetBit(i):
            arr[i] = 1
    return arr

def main():
    # 1. Load your CSV dataset (adjust filename/column names as needed)
    file_path = 'C:/Users/gagno/Downloads/molecule_dataset.csv'
    df = pd.read_csv(file_path)
    print(df.head(18))  # Quick look at data
    
    # 2. Basic cleaning
    df = df.replace([np.inf, -np.inf], np.nan)
    df["Enhancement Factor"] = pd.to_numeric(df["Enhancement Factor"], errors='coerce')
    
    # Drop rows with missing values in 'Smiles' or 'Enhancement Factor'
    df.dropna(subset=["Smiles", "Enhancement Factor"], inplace=True)
    
    # 3. Convert SMILES to numeric features (256-bit Morgan fingerprints)
    smiles_list = df["Smiles"].astype(str).tolist()
    enhancement_factors = df["Enhancement Factor"].values
    
    X_fps = []
    valid_y = []
    for smi, val in zip(smiles_list, enhancement_factors):
        fp = smiles_to_morgan_fp(smi, radius=2, n_bits=256)
        if fp is not None:
            X_fps.append(fp)
            valid_y.append(val)
    
    X = np.array(X_fps, dtype=float)
    y = np.array(valid_y, dtype=float)
    
    # --- NEW: Log-transform the Enhancement Factor ---
    # If EF is always > 0, np.log1p can drastically reduce MSE.
    y_log = np.log1p(y)  # log(1 + EF)
    
    # 4. Train-test split
    X_train, X_test, y_train_log, y_test_log = train_test_split(
        X, y_log, test_size=0.2, random_state=42
    )
    
    # 5. Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    
    # 6. Define and train a neural network (MLPRegressor)
    nn_model = MLPRegressor(
        hidden_layer_sizes=(16, 8),  # Example architecture
        activation='relu',
        solver='adam',
        max_iter=2000,
        random_state=42
    )
    nn_model.fit(X_train_scaled, y_train_log)
    
    # 7. Evaluate on the test set
    #    Predictions are in log space, so exponentiate back
    y_pred_log = nn_model.predict(X_test_scaled)
    y_pred = np.expm1(y_pred_log)  # revert from log(EF) to EF
    
    # Convert y_test_log back to original EF scale
    y_test = np.expm1(y_test_log)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Adjusted R²
    n = len(y_test)
    p = X_test_scaled.shape[1]
    if n > p + 1:
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    else:
        adjusted_r2 = float('nan')
    
    print("\n=== Neural Network Model (Test Set) ===")
    print("Log-transform of Enhancement Factor + 256-bit fingerprints")
    print("Architecture: hidden_layer_sizes=(16, 8), activation='relu'")
    print(f"MSE  : {mse:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"MAE  : {mae:.4f}")
    print(f"R²   : {r2:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    
    # 8. Interactive input for new SMILES outside the dataset
    print("\n=== Predict Enhancement Factor for a new SMILES ===")
    while True:
        user_input = input("Enter a SMILES string (or type 'quit' to exit): ").strip()
        if user_input.lower() == 'quit':
            print("Exiting.")
            break
        
        new_fp = smiles_to_morgan_fp(user_input, radius=2, n_bits=256)
        if new_fp is None:
            print("Invalid SMILES entered. Please try again.\n")
            continue
        
        # Scale the new fingerprint
        new_fp_scaled = scaler.transform([new_fp])
        # Predict in log space, then exponentiate
        pred_log = nn_model.predict(new_fp_scaled)[0]
        pred_val = np.expm1(pred_log)
        print(f"Predicted Enhancement Factor: {pred_val:.4f}\n")

if __name__ == "__main__":
    main()


    ID                         Name  \
0     1         Ibuprofen / HP-B-CD   
1     2      Dexamethasone / B - CD   
2     3         Piroxicam / HP-B-CD   
3     4    Hydrocortisone / HP-B-CD   
4     5       Albendazole / RM-B-CD   
5     6            Valtarsan / B-CD   
6     7        Nifedipine / HP-B-CD   
7     8        Nimodipine / HP-B-CD   
8     9     Carbamazepine / HP-B-CD   
9    10      Itraconazole / HP-B-CD   
10   11       Simvastatin / HP-B-CD   
11   12          Curcumin / HP-B-CD   
12   13  Indomethacin / HP- B - CD    
13   14     Naproxen / HP - B - CD    
14   15        Diclofenac / HP-B-CD   
15   16   Resveratrol / HP-Gamma CD   
16   17              Amphotericin B   
17   18                 Budesonide    

                                               Smiles MolWeight  LogP  \
0                          CC(C)Cc1ccc(cc1)C(C)C(=O)O   1606.29  3.84   
1   CC(C)C1C(=O)CC2C(C1O)(CC(C1=CC(=O)C=C2C1(C)O)F...   1527.47  1.93   
2                     CN1NS(=O)(=O)C2=C

[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerator
[01:36:54] DEPRECATION WARNING: please use MorganGenerat


=== Neural Network Model (Test Set) ===
Log-transform of Enhancement Factor + 256-bit fingerprints
Architecture: hidden_layer_sizes=(16, 8), activation='relu'
MSE  : 147778.6841
RMSE : 384.4199
MAE  : 159.6996
R²   : -0.2065
Adjusted R²: nan

=== Predict Enhancement Factor for a new SMILES ===


Below is a siginifacnly improved model
=
=
=
=
=
=
=
=


In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam

# RDKit for molecular descriptors
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski, MACCSkeys

# Train-test split, metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def smiles_to_morgan_fp(smi, radius=3, n_bits=2048):
    """
    Convert a SMILES string to a Morgan fingerprint with higher bit count
    for better representation of structural features
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    for i in range(n_bits):
        if fp.GetBit(i):
            arr[i] = 1
    return arr

def smiles_to_maccs(smi):
    """
    Convert SMILES to MACCS keys (166-bit fingerprint)
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    fp = MACCSkeys.GenMACCSKeys(mol)
    arr = np.zeros((167,), dtype=int)  # MACCS has 167 bits
    for i in range(167):
        if fp.GetBit(i):
            arr[i] = 1
    return arr

def calculate_molecular_descriptors(smi):
    """
    Calculate additional molecular descriptors to capture physicochemical properties
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    
    descriptors = []
    
    # Basic properties
    descriptors.append(Descriptors.MolWt(mol))                 # Molecular weight
    descriptors.append(Descriptors.MolLogP(mol))               # LogP
    descriptors.append(Descriptors.TPSA(mol))                  # Topological polar surface area
    descriptors.append(Descriptors.NumHDonors(mol))            # Number of H-bond donors
    descriptors.append(Descriptors.NumHAcceptors(mol))         # Number of H-bond acceptors
    descriptors.append(Descriptors.NumRotatableBonds(mol))     # Number of rotatable bonds
    descriptors.append(Descriptors.NumAromaticRings(mol))      # Number of aromatic rings
    descriptors.append(Descriptors.NumAliphaticRings(mol))     # Number of aliphatic rings
    descriptors.append(Descriptors.NumSaturatedRings(mol))     # Number of saturated rings
    descriptors.append(Descriptors.FractionCSP3(mol))          # Fraction of C atoms that are sp3 hybridized
    descriptors.append(Descriptors.NumHeteroatoms(mol))        # Number of heteroatoms
    descriptors.append(Descriptors.HeavyAtomCount(mol))        # Number of heavy atoms
    descriptors.append(Descriptors.NumValenceElectrons(mol))   # Number of valence electrons
    descriptors.append(Descriptors.BalabanJ(mol))              # Balaban's J value
    descriptors.append(Descriptors.BertzCT(mol))               # Bertz CT
    
    # Additional descriptors
    descriptors.append(Descriptors.Chi0n(mol))                 # Connectivity index
    descriptors.append(Descriptors.Chi0v(mol))                 # Valence connectivity index
    descriptors.append(Descriptors.HallKierAlpha(mol))         # Hall-Kier alpha value
    descriptors.append(Descriptors.Kappa1(mol))                # Kappa shape index
    descriptors.append(Descriptors.LabuteASA(mol))             # Labute ASA

    return descriptors

def create_nn_model(input_dim, dropout_rate=0.3, l1_reg=0.001, l2_reg=0.001):
    """
    Create a neural network model with regularization to prevent overfitting
    """
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,), 
              kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        BatchNormalization(),
        
        Dense(1)  # Output layer (no activation for regression)
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )
    
    return model

def create_ensemble_model(input_dim):
    """
    Create an ensemble of sub-models with different architectures
    """
    # Base model inputs
    inputs = Input(shape=(input_dim,))
    
    # Sub-model 1: Deep network with more regularization
    x1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001))(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.4)(x1)
    x1 = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.4)(x1)
    x1 = Dense(32, activation='relu')(x1)
    x1 = Dense(1, name='model1_output')(x1)
    
    # Sub-model 2: Shallower network with less regularization
    x2 = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.0005, l2=0.0005))(inputs)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Dense(32, activation='relu')(x2)
    x2 = Dense(1, name='model2_output')(x2)
    
    # Sub-model 3: Very shallow network
    x3 = Dense(32, activation='relu')(inputs)
    x3 = Dense(16, activation='relu')(x3)
    x3 = Dense(1, name='model3_output')(x3)
    
    # Combine predictions (average)
    outputs = tf.keras.layers.Average()([x1, x2, x3])
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )
    
    return model

def visualize_predictions(y_true, y_pred, title):
    """
    Create a visualization of predictions vs actual values
    """
    plt.figure(figsize=(10, 6))
    
    # Create a scatter plot
    plt.scatter(y_true, y_pred, alpha=0.7)
    
    # Add a perfect prediction line
    max_val = max(np.max(y_true), np.max(y_pred))
    min_val = min(np.min(y_true), np.min(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    # Calculate and display R² value
    r2 = r2_score(y_true, y_pred)
    plt.text(0.05, 0.95, f'R² = {r2:.4f}', transform=plt.gca().transAxes, 
             fontsize=12, verticalalignment='top')
    
    # Labels and title
    plt.xlabel('Actual Enhancement Factor')
    plt.ylabel('Predicted Enhancement Factor')
    plt.title(title)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ', '_')}.png")
    plt.close()

def main():
    # Load dataset
    print("Loading dataset...")
    file_path = 'C:/Users/gagno/Downloads/molecule_dataset.csv'
    df = pd.read_csv(file_path)
    
    # Data cleaning
    df = df.replace([np.inf, -np.inf], np.nan)
    df["Enhancement Factor"] = pd.to_numeric(df["Enhancement Factor"], errors='coerce')
    df.dropna(subset=["Smiles", "Enhancement Factor"], inplace=True)
    
    print(f"Dataset loaded. Shape: {df.shape}")
    print(f"Enhancement Factor range: {df['Enhancement Factor'].min()} to {df['Enhancement Factor'].max()}")
    
    # Extract SMILES and target values
    smiles_list = df["Smiles"].astype(str).tolist()
    enhancement_factors = df["Enhancement Factor"].values
    
    # Log-transform target due to wide range
    enhancement_factors_log = np.log1p(enhancement_factors)
    
    # Feature extraction
    print("Extracting molecular features...")
    features_list = []
    valid_indices = []
    
    for i, smi in enumerate(smiles_list):
        # Get Morgan fingerprint (2048 bits, radius 3)
        morgan_fp = smiles_to_morgan_fp(smi, radius=3, n_bits=2048)
        
        # Get MACCS keys fingerprint (167 bits)
        maccs_fp = smiles_to_maccs(smi)
        
        # Get descriptors
        descriptors = calculate_molecular_descriptors(smi)
        
        if morgan_fp is not None and maccs_fp is not None and descriptors is not None:
            # Combine all features
            combined_features = np.concatenate([morgan_fp, maccs_fp, descriptors])
            features_list.append(combined_features)
            valid_indices.append(i)
    
    # Convert to numpy arrays
    X = np.array(features_list)
    y = enhancement_factors_log[valid_indices]  # Using log-transformed target
    y_original = enhancement_factors[valid_indices]  # Original values for evaluation
    
    print(f"Features extracted. X shape: {X.shape}")
    
    # Remove near-zero variance features
    variance = np.var(X, axis=0)
    keep_indices = np.where(variance > 0.01)[0]
    X = X[:, keep_indices]
    print(f"After removing low-variance features: {X.shape}")
    
    # Feature selection with Random Forest
    print("Performing feature selection...")
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    # Select features based on importance
    selector = SelectFromModel(rf, threshold="mean", prefit=True)
    X_selected = selector.transform(X)
    print(f"After feature selection: {X_selected.shape}")
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    
    # Bin the target for stratified sampling
    y_bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
    
    # Split data with stratification
    X_train, X_test, y_train, y_test, y_orig_train, y_orig_test = train_test_split(
        X_scaled, y, y_original, test_size=0.2, random_state=42, stratify=y_bins
    )
    
    print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")
    
    # K-fold cross-validation to ensure robustness
    print("Performing cross-validation...")
    k_fold = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(k_fold.split(X_train)):
        print(f"\nFold {fold+1}/5")
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Create and train ensemble model
        model = create_ensemble_model(X_fold_train.shape[1])
        
        # Callbacks
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=30,
            restore_best_weights=True,
            verbose=1
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=0.00001,
            verbose=1
        )
        
        # Train model
        history = model.fit(
            X_fold_train, y_fold_train,
            epochs=200,
            batch_size=8,  # Small batch size for small dataset
            validation_data=(X_fold_val, y_fold_val),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
        
        # Evaluate
        y_fold_pred = model.predict(X_fold_val).flatten()
        fold_r2 = r2_score(y_fold_val, y_fold_pred)
        cv_scores.append(fold_r2)
        print(f"Fold {fold+1} R²: {fold_r2:.4f}")
    
    print(f"\nCross-validation R² scores: {cv_scores}")
    print(f"Mean CV R²: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
    
    # Train final model on all training data
    print("\nTraining final model...")
    final_model = create_ensemble_model(X_train.shape[1])
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=50,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=15,
        min_lr=0.00001,
        verbose=1
    )
    
    # Train with validation split for early stopping
    history = final_model.fit(
        X_train, y_train,
        epochs=300,
        batch_size=8,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()
    
    # Evaluate on test set
    y_pred_log = final_model.predict(X_test).flatten()
    
    # Convert predictions back to original scale
    y_pred = np.expm1(y_pred_log)
    
    # Calculate metrics on log scale
    mse_log = mean_squared_error(y_test, y_pred_log)
    rmse_log = np.sqrt(mse_log)
    mae_log = mean_absolute_error(y_test, y_pred_log)
    r2_log = r2_score(y_test, y_pred_log)
    
    # Calculate metrics on original scale
    mse = mean_squared_error(y_orig_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_orig_test, y_pred)
    r2 = r2_score(y_orig_test, y_pred)
    
    # Calculate adjusted R²
    n = len(y_test)
    p = X_test.shape[1]
    if n > p + 1:
        adjusted_r2_log = 1 - (1 - r2_log) * (n - 1) / (n - p - 1)
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    else:
        adjusted_r2_log = adjusted_r2 = float('nan')
    
    # Print results
    print("\n=== Neural Network Ensemble Results (Test Set) ===")
    print("Log-transformed space:")
    print(f"MSE  : {mse_log:.4f}")
    print(f"RMSE : {rmse_log:.4f}")
    print(f"MAE  : {mae_log:.4f}")
    print(f"R²   : {r2_log:.4f}")
    print(f"Adjusted R²: {adjusted_r2_log:.4f}")
    
    print("\nOriginal space:")
    print(f"MSE  : {mse:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"MAE  : {mae:.4f}")
    print(f"R²   : {r2:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    
    # Visualize predictions
    visualize_predictions(y_orig_test, y_pred, "Neural Network: Actual vs Predicted")
    
    # Save final model for future use
    final_model.save('enhancement_factor_model.h5')
    
    # Save feature processing pipeline
    import pickle
    with open('model_preprocessing.pkl', 'wb') as f:
        pickle.dump({
            'scaler': scaler,
            'feature_selector': selector,
            'keep_indices': keep_indices
        }, f)
    
    # Interactive prediction loop
    print("\n=== Predict Enhancement Factor for new SMILES ===")
    while True:
        user_input = input("Enter a SMILES string (or 'quit' to exit): ").strip()
        if user_input.lower() == 'quit':
            print("Exiting.")
            break
        
        # Extract features
        morgan_fp = smiles_to_morgan_fp(user_input, radius=3, n_bits=2048)
        maccs_fp = smiles_to_maccs(user_input)
        descriptors = calculate_molecular_descriptors(user_input)
        
        if morgan_fp is None or maccs_fp is None or descriptors is None:
            print("Invalid SMILES entered. Please try again.\n")
            continue
        
        # Combine features
        combined_features = np.concatenate([morgan_fp, maccs_fp, descriptors])
        
        # Apply same preprocessing steps
        features = combined_features[keep_indices]  # Apply variance filter
        features = selector.transform([features])  # Apply feature selection
        features_scaled = scaler.transform(features)  # Apply scaling
        
        # Predict
        pred_log = final_model.predict(features_scaled).flatten()[0]
        pred_val = np.expm1(pred_log)
        
        print(f"Predicted Enhancement Factor: {pred_val:.4f}\n")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'tensorflow'